In [1]:
# Load packages
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
path = 'C:/Users/angel/Documents/Economics/Research/Banking Project/data/intermediate/call_reports_SP/'

# Set path to be the directory:
os.chdir(path)

In [ ]:
#df = pd.concat([pd.read_csv(f) for f in os.listdir() if f.endswith('.csv')])
#df = df.iloc[:, 0:270]
#df['Date'] = df['Date'].astype(str)
#df['Month'] = np.where(df['Date'].str.len() == 8, df['Date'].str[0:2], '0'+df['Date'].str[0])
#df['Day'] = df['Date'].str[-6:-4]
#df['Year'] = df['Date'].str[-4:]
#df['Date'] = pd.to_datetime(df['Year'] + '-' + df['Month'] + '-' + df['Day'])
#df.to_csv(path + 'call_reports.csv', index = False)

In [3]:
# Read file that contains only the variables of interest, specify that the column 'Date' is a date:
df = pd.read_csv('call_reports.csv', parse_dates = ['Date'])

C:\Users\angel\AppData\Local\Temp\ipykernel_12152\1849562282.py:2: DtypeWarning: Columns (102,146,187,228) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('call_reports.csv', parse_dates = ['Date'])


In [ ]:
df.columns

### Deposits

In [4]:
# Create a new column that will contain the deposit expenditure it will sum 'RIADA517', 'RIADA518', 'RIAD4508', 'RIAD0093' for
# any date before 2017:
df['Deposit Expenditure'] = np.where(df['Date'] < '2017-01-01', df['RIADA517']+df['RIADA518']+df['RIAD4508']+df['RIAD0093'], 
                                     df['RIADHK03']+df['RIADHK04']+df['RIAD4508']+df['RIAD0093'])

In [5]:
df['Total Deposit'] = np.where(df['Date'] < '2017-01-01', df['RCON3485']+df['RCONB563']+df['RCONA514']+df['RCONA529'], 
                                     df['RCON3485']+df['RCONB563']+df['RCONHK16']+df['RCONHK17'])

In [6]:
df['Deposit Rate'] = df['Deposit Expenditure'] / df['Total Deposit']

In [7]:
cr_subset = df[['Date', 'IDRSSD', 'Total Deposit', 'Deposit Expenditure', 'Deposit Rate']]
cr_subset.to_csv('call_reports_subset.csv', index = False)

In [ ]:
df = df[df['Total Deposit']>0]

In [ ]:
# Create an extra column in df3 with the avg. deposit rate per Date:
df['Avg. Deposit Rate'] = df.groupby('Date')['Deposit Rate'].transform('mean')
df['Std. Deposit Rate'] = df.groupby('Date')['Deposit Rate'].transform('std')

In [ ]:
df['R_hat'] = (df['Deposit Rate'] - df['Avg. Deposit Rate']) / df['Std. Deposit Rate']

In [ ]:
# show the series of avg. deposit rate per Date:
df['Avg. Deposit Rate'].unique()

In [ ]:
df['Std. Deposit Rate'].unique()

### Deposits

In [ ]:
# Until here, dataset matches with Lucas.
#df2[(df2['Date']>'2023-01-01') & (df2['Date']<'2024-01-01')]['Deposit Rate'].describe()

In [ ]:
# Trim the 'Deposit Rate' from df2 on the 1st and 99th percentiles, per year:
df2 = df.groupby('Date').apply(lambda x: x[(x['Deposit Rate'] >= x['Deposit Rate'].quantile(0.01, interpolation='midpoint')) & 
                                            (x['Deposit Rate'] <= x['Deposit Rate'].quantile(0.99, interpolation='midpoint'))]).reset_index(drop=True)

# We may not want to trim since we don't want BofA to be dropped out.

In [ ]:
# plot the kernel distribution of the deposit rates for the above dates:
sns.kdeplot(df[df['Date']=='2001-03-31']['R_hat'], label='2001-03-31', color='black', fill=True, alpha=0.5)
sns.kdeplot(df[df['Date']=='2022-03-31']['R_hat'], label='2022-03-31', color='lightblue', fill=True, alpha=0.5)
plt.legend()
plt.show()

In [ ]:
df_bottom = df.groupby('Date').apply(lambda x: x[x['Total Deposit'] <= x['Total Deposit'].quantile(0.1)]).reset_index(drop=True)

# plot the kernel distribution of the deposit rates for the above dates:
sns.kdeplot(df_bottom[df_bottom['Date']=='2001-03-31']['R_hat'], label='2001-03-31', color='black', fill=True, alpha=0.5)
sns.kdeplot(df_bottom[df_bottom['Date']=='2022-03-31']['R_hat'], label='2022-03-31', color='lightblue', fill=True, alpha=0.5)
plt.legend()
plt.show()

In [ ]:
df_top = df.groupby('Date').apply(lambda x: x[x['Total Deposit'] >= x['Total Deposit'].quantile(0.9)]).reset_index(drop=True)

# plot the kernel distribution of the deposit rates for the above dates:
sns.kdeplot(df_top[df_top['Date']=='2001-03-31']['R_hat'], label='2001-03-31', color='black', fill=True, alpha=0.5)
sns.kdeplot(df_top[df_top['Date']=='2022-03-31']['R_hat'], label='2022-03-31', color='lightblue', fill=True, alpha=0.5)
plt.legend()
plt.show()

### Loans and Assets

In [ ]:
# Check total Interest Income:
df['Interest Income'] = df['RIAD4107']
# Check Total Assets:
df['Total Assets'] = df['RCFD2170']
df['ROA'] = df['Interest Income'] / df['Total Assets']

In [ ]:
df = df[df['Total Assets']>0]

In [ ]:
# Create an extra column in df3 with the avg. deposit rate per Date:
df['Avg. ROA'] = df.groupby('Date')['ROA'].transform('mean')
df['Std. ROA'] = df.groupby('Date')['ROA'].transform('std')

In [ ]:
df['ROA_hat'] = (df['ROA'] - df['Avg. ROA']) / df['Std. ROA']

In [ ]:
# plot the kernel distribution of the ROA for the above dates:
sns.kdeplot(df[df['Date']=='2001-03-31']['ROA_hat'], label='2001', color='black', fill=True, alpha=0.5)
sns.kdeplot(df[df['Date']=='2022-03-31']['ROA_hat'], label='2022', color='lightblue', fill=True, alpha=0.5)
plt.legend()
plt.show()